# Representation Learning Assignment
This assignment covers key topics in representation learning using PyTorch datasets. You will implement tasks for Contrastive Learning, Energy-Based Models. Use torchvision.datasets.MNIST for Exercises 1, 2.

**Total Points: 12**
- Exercise 1: Contrastive Learning (8 points)
- Exercise 2: Energy-Based Models (4 points)

Import necessary libraries and load datasets as needed.


In [1]:
# --- Install dependencies (CPU-only build by default) ---
!pip install torch torchvision --extra-index-url https://download.pytorch.org/whl/cpu
!pip install -q torch_geometric

# --- Imports ---
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import Subset, DataLoader

print("Torch:", torch.__version__, "Torchvision:", torchvision.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu
     -------------------------------------- 619.3/619.3 MB 2.6 MB/s eta 0:00:00
     ---------------------------------------- 1.6/1.6 MB 3.8 MB/s eta 0:00:00
  Using cached filelock-3.19.1-py3-none-any.whl (15 kB)
  Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
  Using cached https://download.pytorch.org/whl/networkx-3.2.1-py3-none-any.whl (1.6 MB)
     ------------------------------------ 199.3/199.3 KB 756.2 kB/s eta 0:00:00
  Using cached https://download.pytorch.org/whl/mpmath-1.3.0-py3-none-any.whl (536 kB)


You should consider upgrading via the 'C:\Users\acer\Desktop\fuse_ml\env\Scripts\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\acer\Desktop\fuse_ml\env\Scripts\python.exe -m pip install --upgrade pip' command.


Torch: 2.8.0+cpu Torchvision: 0.23.0+cpu
Using device: cpu


In [2]:
# --- MNIST (for Exercises 1, 2) ---
mnist_transform = transforms.ToTensor()

mnist_train_full = datasets.MNIST(
    root="./data", train=True, download=True, transform=mnist_transform
)
mnist_test_full = datasets.MNIST(
    root="./data", train=False, download=True, transform=mnist_transform
)

# Smaller subsets for quicker experiments
train_indices = list(range(1000))
test_indices = list(range(200))

mnist_train = Subset(mnist_train_full, train_indices)
mnist_test = Subset(mnist_test_full, test_indices)

print(f"MNIST Train Subset: {len(mnist_train)} samples")
print(f"MNIST Test Subset: {len(mnist_test)} samples")

100%|██████████| 9.91M/9.91M [00:17<00:00, 552kB/s] 
100%|██████████| 28.9k/28.9k [00:00<00:00, 60.2kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 963kB/s] 
100%|██████████| 4.54k/4.54k [00:00<00:00, 2.24MB/s]

MNIST Train Subset: 1000 samples
MNIST Test Subset: 200 samples


## Exercise 1: Contrastive Learning (8 points)

We’ll simulate contrastive learning on MNIST images by computing similarities, pairs, and losses.

### Task 1a (1 point): Cosine Similarity

Compute row-wise cosine similarity between two batches of embeddings.

In [3]:
### Ex-1-Task-1
import torch
import torch.nn.functional as F

def compute_similarity(emb1, emb2):
    """
    emb1, emb2: (B, D)
    Returns: (B,) cosine similarities
    """
    ### BEGIN SOLUTION 
    # YOUR CODE HERE
    return F.cosine_similarity(emb1, emb2, dim=1)
    ### END SOLUTION

# Test
x = torch.randn(5, 10)
y = x.clone()
print(compute_similarity(x, y))

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [4]:
# INTENTIONALLY LEFT BLANK

### Task 1b: Positive & Negative Pairs (2 points)

Generate positive pairs (same sample) and negative pairs (different sample) for a batch.
Simulate negatives by randomly shuffling batch.


In [5]:
### Ex-1-Task-2
def generate_pairs(batch):
    """
    batch: (B, D)
    Returns:
        pos: (B, D)
        neg: (B, D)
    """
    ### BEGIN SOLUTION 
    # YOUR CODE HERE
    pos = batch.clone()  # positive pairs are identical
    neg = batch[torch.randperm(batch.size(0))]  # shuffle to get negatives
    return pos, neg

    ### END SOLUTION

# Test
batch = torch.randn(5, 8)
pos, neg = generate_pairs(batch)
print("Positive pairs:\n", pos)
print("Negative pairs:\n", neg)

Positive pairs:
 tensor([[ 0.7708, -0.0480, -0.1760, -0.4657,  0.9167, -1.1371,  0.7879,  0.4465],
        [-1.2869, -0.9598,  1.4369, -0.1307,  0.8614, -1.5900, -1.5318,  0.4548],
        [-0.6161, -0.0787,  0.0724, -0.5983,  1.0229,  1.0371, -0.5322,  0.1165],
        [ 0.0957, -0.6715,  0.0714,  0.8831,  0.5460,  0.8037,  1.3451, -1.2603],
        [ 1.3135, -0.2352,  0.4123,  1.5636, -0.9952,  0.0932, -2.3883,  0.6086]])
Negative pairs:
 tensor([[-0.6161, -0.0787,  0.0724, -0.5983,  1.0229,  1.0371, -0.5322,  0.1165],
        [-1.2869, -0.9598,  1.4369, -0.1307,  0.8614, -1.5900, -1.5318,  0.4548],
        [ 1.3135, -0.2352,  0.4123,  1.5636, -0.9952,  0.0932, -2.3883,  0.6086],
        [ 0.7708, -0.0480, -0.1760, -0.4657,  0.9167, -1.1371,  0.7879,  0.4465],
        [ 0.0957, -0.6715,  0.0714,  0.8831,  0.5460,  0.8037,  1.3451, -1.2603]])


In [6]:
# INTENTIONALLY LEFT BLANK

### Task 1c:NT-Xent Loss (2 points)

Implement normalized temperature-scaled cross-entropy loss for a batch.

In [7]:
### Ex-1-Task-3
import torch
import torch.nn.functional as F

def nt_xent_loss(sims, temp=0.5):
    """
    sims: (B, B) similarity matrix
    temp: scalar temperature
    Returns: scalar loss
    """
    ### BEGIN SOLUTION 
    # YOUR CODE HERE
    # sims / temperature
    sims = sims / temp
    # For each sample, the positive pair is on the diagonal
    labels = torch.arange(sims.size(0)).to(sims.device)
    
    # Use cross entropy: treat each row as logits
    loss = F.cross_entropy(sims, labels)
    return loss
    ### END SOLUTION

# Test
sims = torch.randn(3, 3)
print("NT-Xent loss:", nt_xent_loss(sims))

NT-Xent loss: tensor(0.6327)


In [8]:
# Quick visible test with identity similarity
sims = torch.eye(4)  # 4x4 identity matrix
loss = nt_xent_loss(sims)
print("NT-Xent loss:", loss.item())  # Should be > 0


NT-Xent loss: 0.3407530188560486


In [9]:
# INTENTIONALLY LEFT BLANK


### Task 1d: Augment MNIST Image (2 points)

Apply random small rotation ±10° to a single MNIST image.


In [10]:
### Ex-1-Task-4
from torchvision import transforms
from torchvision.transforms import ToPILImage, ToTensor
augment = transforms.RandomRotation(degrees=10)
to_pil = ToPILImage()
to_tensor = ToTensor()
def augment_image(img):
    """
    img: (28, 28) tensor
    Returns: augmented image (28,28)
    """
    ### BEGIN SOLUTION 
    # YOUR CODE HERE
    ### BEGIN SOLUTION
    pil_img = to_pil(img)          # convert tensor to PIL
    aug_img = augment(pil_img)     # apply random rotation
    return to_tensor(aug_img)      # convert back to tensor
    ### END SOLUTION

In [11]:
img, label = mnist_train[0]
aug_img = augment_image(img)
print("Original shape:", img.shape)
print("Augmented shape:", aug_img.shape)

Original shape: torch.Size([1, 28, 28])
Augmented shape: torch.Size([1, 28, 28])


In [12]:
# INTENTIONALLY LEFT BLANK

### Task 1e: Contrastive Embedding Distance (1 point)

Compute Euclidean distance between two batches of embeddings row-wise.

In [13]:
### Ex-1-Task-5
import torch

def embedding_distance(emb1, emb2):
    """
    emb1, emb2: (B, D)
    Returns: (B,) Euclidean distances
    """
    ### BEGIN SOLUTION 
    # YOUR CODE HERE
    return torch.norm(emb1 - emb2, dim=1)
    ### END SOLUTION

# --- Test Example ---
batch_size, dim = 4, 5
emb1 = torch.randn(batch_size, dim)
emb2 = torch.randn(batch_size, dim)

distances = embedding_distance(emb1, emb2)
print("Embeddings 1:\n", emb1)
print("Embeddings 2:\n", emb2)
print("Row-wise Euclidean distances:\n", distances)

Embeddings 1:
 tensor([[ 0.0978,  1.9510,  0.2993,  2.2163,  0.3813],
        [ 0.4847,  0.6972,  0.0220, -0.7370, -0.9647],
        [-1.9789, -0.1925,  0.3206,  0.9641,  0.2400],
        [-0.1766,  0.8947,  2.1024, -1.2369,  1.2713]])
Embeddings 2:
 tensor([[-2.0376,  0.5602,  0.8938, -0.8978,  1.5713],
        [ 0.3051, -0.2302,  0.3001,  1.4844,  1.7399],
        [ 0.7667,  1.5337,  0.6802,  1.2916,  0.1828],
        [-2.0865, -1.5948,  2.0347, -0.6210,  0.3169]])
Row-wise Euclidean distances:
 tensor([4.2381, 3.6358, 3.2800, 3.3377])


In [14]:
# INTENTIONALLY LEFT BLANK

## Exercise 2: Energy-Based Models (4 points)



### Task 2a (2 points): Define Energy Function

Energy function for binary classification: E(x,y;w)=−y⋅(w⋅x)

In [15]:
### Ex-2-Task-1
def energy_function(x, y, w):
    """
    x: (784,), y: +1/-1, w: (784,)
    Returns: scalar energy
    """
    ### BEGIN SOLUTION 
    # YOUR CODE HERE
    return -y * torch.dot(w, x)
    ### END SOLUTION

# Test
x = torch.randn(784)
w = torch.randn(784)
y = 1
print("Energy:", energy_function(x, y, w))

Energy: tensor(-2.7599)


In [16]:
# INTENTIONALLY LEFT BLANK

### Task 2b (2 points): Perceptron Loss (2 point)

Compute perceptron loss: max(0, E(x, y_true) - E(x, y_pred))

In [17]:
### Ex-2-Task-2
def perceptron_loss(x, y_true, y_pred, w):
    """
    x: (784,), y_true, y_pred: +1/-1
    w: (784,)
    Returns: scalar loss
    """
    ### BEGIN SOLUTION 
    # YOUR CODE HERE
    energy_true = energy_function(x, y_true, w)
    energy_pred = energy_function(x, y_pred, w)
    return torch.clamp(energy_true - energy_pred, min=0.0)
    ### END SOLUTION

# Test
print("Perceptron loss:", perceptron_loss(x, 1, -1, w))

Perceptron loss: tensor(0.)


In [18]:
# INTENTIONALLY LEFT BLANK